### Config graph:

In [138]:
import pandas as pd

In [139]:
nodes = pd.read_csv("nodes.csv",index_col=0)
nodes

,name,type,Latitude,Longitude,value
index,,,,,
1,Helsinki,demand,60.192059,24.945831,1765.110907
2,Lahti,demand,60.985588,25.661613,174.788535
3,Turku,demand,60.454510,22.264824,381.930099
4,Tampere,demand,61.500678,23.756254,475.878805
5,Hämeenlinna,demand,61.025357,24.465736,74.178161
6,Hyvinkää,demand,60.634925,24.860155,63.533291
7,Riihimäki,demand,60.738222,24.754285,42.089081
8,Lohja,demand,60.293841,24.048630,50.215955
9,Pori,demand,61.516220,21.807122,126.637667


In [140]:
edges = pd.read_csv("edges.csv")
edges

,i,j,voltage,length,bundle
0,8,6,400,68,2
1,8,1,400,57,2
2,2,7,110,62,1
3,5,6,400,49,2
4,4,5,400,77,2
5,7,6,400,13,2
6,5,2,110,73,1
7,4,10,400,143,2
8,10,8,400,205,2
9,10,3,400,91,2


### Solve

In [149]:
from pulp import *

Initialize problem as a minimizing problem:

In [166]:
prob = LpProblem("Problem", LpMinimize)

Define variables:

In [167]:
x = LpVariable.dicts("Power_Line", edges.index,lowBound=0)

Define cost function:

In [168]:
prob += lpSum([x[i] * edges['length'][i] / edges['voltage'][i] for i in edges.index])

In [169]:
for i in edges.index:
    prob += x[i] <= 300

Define constraints:

In [170]:
for i in nodes.index:
    if nodes['type'][i] == 'demand':
        prob += (
            lpSum([x[n] for n in edges[edges['j'] == i].index]) -
            lpSum([x[n] for n in edges[edges['i'] == i].index]) 
            == nodes['value'][i]
        )
    elif nodes['type'][i] == 'source':
        prob += (
            lpSum([x[n] for n in edges[edges['i'] == i].index]) -
            lpSum([x[n] for n in edges[edges['j'] == i].index]) 
            <= nodes['value'][i]
        )
        

Print total problem description:

In [171]:
prob

Problem:
MINIMIZE
0.17*Power_Line_0 + 0.1425*Power_Line_1 + 0.63*Power_Line_10 + 0.295*Power_Line_11 + 0.145*Power_Line_12 + 0.28*Power_Line_13 + 0.1275*Power_Line_14 + 0.17*Power_Line_15 + 0.1425*Power_Line_16 + 0.5636363636363636*Power_Line_17 + 0.1225*Power_Line_18 + 0.1925*Power_Line_19 + 0.5636363636363636*Power_Line_2 + 0.0325*Power_Line_20 + 0.6636363636363637*Power_Line_21 + 0.3575*Power_Line_22 + 0.5125*Power_Line_23 + 0.2275*Power_Line_24 + 0.63*Power_Line_25 + 0.295*Power_Line_26 + 0.145*Power_Line_27 + 0.28*Power_Line_28 + 0.1275*Power_Line_29 + 0.1225*Power_Line_3 + 0.0925*Power_Line_30 + 0.0925*Power_Line_31 + 1.2775*Power_Line_32 + 0.215*Power_Line_33 + 1.39*Power_Line_34 + 1.6*Power_Line_35 + 2.2454545454545456*Power_Line_36 + 0.0625*Power_Line_37 + 0.3225*Power_Line_38 + 1.34*Power_Line_39 + 0.1925*Power_Line_4 + 0.1*Power_Line_40 + 0.2*Power_Line_41 + 0.2*Power_Line_42 + 0.11818181818181818*Power_Line_43 + 2.1363636363636362*Power_Line_44 + 0.2727272727272727*Power_Li

Solve problem:

In [172]:
prob.solve()
prob.status

1

In [173]:
for i in edges.index:
    if value(x[i]) == 0:
        continue
    inode = nodes.loc[edges['i'][i]]
    jnode = nodes.loc[edges['j'][i]]
    print(f"{inode['name']}({inode['type']})", 'to', f"{jnode['name']}({jnode['type']})", ":", value(x[i]), "MW")

Hämeenlinna(demand) to Hyvinkää(demand) : 11.719004 MW
Tampere(demand) to Hämeenlinna(demand) : 260.6857 MW
Hämeenlinna(demand) to Lahti(demand) : 174.78854 MW
Rauma(demand) to Lohja(demand) : 50.215955 MW
Rauma(demand) to Turku(demand) : 125.9301 MW
Helsinki(demand) to Hyvinkää(demand) : 93.903367 MW
Hyvinkää(demand) to Riihimäki(demand) : 42.08908 MW
Rauma(demand) to Tampere(demand) : 73.564506 MW
Pori(demand) to Tampere(demand) : 300.0 MW
Pori(demand) to Rauma(demand) : 300.0 MW
Olkiluoto 1(source) to Pori(demand) : 126.63767 MW
Olkiluoto 2(source) to Pori(demand) : 300.0 MW
Meri-Pori Power Station(source) to Pori(demand) : 300.0 MW
Keljonlahti Power Station(source) to Tampere(demand) : 209.0 MW
PetÌ?jÌ?skoski Dam(source) to Tampere(demand) : 154.0 MW
Naantali Power Station(source) to Turku(demand) : 256.0 MW
Loviisa 1(source) to Helsinki(demand) : 300.0 MW
Loviisa 2(source) to Helsinki(demand) : 300.0 MW
Vuosaari Power Station(source) to Helsinki(demand) : 300.0 MW
Imatra Dam(sourc